# Sample

## Transform

In [ ]:
# # Transformasi
# numerical_transformer = StandardScaler()
# categorical_transformer = Pipeline([
#     ('onehot', OneHotEncoder(handle_unknown = 'ignore', sparse_output = True, max_categories = 50)), 
#     # ('svd', TruncatedSVD(n_components = 100))  # Mengurangi dimensi fitur kategori
# ])
# ordinal_transformer = OrdinalEncoder(categories = oridnal_cat, handle_unknown = 'use_encoded_value', unknown_value = -1)

# # Column Transformer
# prep_stage_2 = ColumnTransformer(
#     transformers = [
#         ("num", numerical_transformer, numeric_cols), 
#         ("cat", categorical_transformer, one_hot_cols), 
#         ("ord", ordinal_transformer, ordinal_cols)
#     ], remainder = "passthrough")

In [ ]:
# # Transform data menggunakan fit_transform
# model_df = prep_stage_2.fit_transform(model_df)

# # Get name column from fit_transform
# model_df = pd.DataFrame(model_df, columns = prep_stage_2.get_feature_names_out())

# # Hilangkan prefix (misalnya, "num__", "cat__", "out__")
# clean_columns = [col.split("__", 1)[-1] for col in model_df.columns]
# model_df.columns = clean_columns

In [ ]:
# # Menampilkan total null pada setiap kolom
# null_columns = model_df.isnull().sum()[model_df.isnull().sum() > 0]
# print(f'Total null columns: {null_columns} \n')

In [ ]:
# # check data type after transform
# model_df.info()

In [ ]:
# # change object after transform
# model_df = convert_object_columns_to_numeric(model_df)
# model_df.info()

In [ ]:
# model_df.head()

## Base Model

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report, confusion_matrix

# # Define pipeline
# base_model_pipe = Pipeline([
#     ("model", RandomForestClassifier(n_estimators = 100, random_state = 42, class_weight = "balanced"))
# ])

# # Cross-validation scores with multiple metrics
# val_score = ["accuracy", "precision", "recall", "f1", "roc_auc"]
# cv_results = cross_validate(base_model_pipe, X_train, y_train, cv = 5, scoring = val_score)
# cv_results

In [ ]:
# print(f"Cross-validation Accuracy: {cv_results['test_accuracy'].mean():.4f}")
# print(f"Cross-validation Precision: {cv_results['test_precision'].mean():.4f}")
# print(f"Cross-validation Recall: {cv_results['test_recall'].mean():.4f}")
# print(f"Cross-validation F1-score: {cv_results['test_f1'].mean():.4f}")
# print(f"Cross-validation AUC-ROC: {cv_results['test_roc_auc'].mean():.4f} \n")

In [ ]:
# # Fit the pipeline
# base_model_pipe.fit(X_train, y_train)

# # Predict
# y_pred = base_model_pipe.predict(X_test)

# # Evaluasi Model
# print(classification_report(y_test, y_pred))

#### Evaluasi Model

In [ ]:
# from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

# # Evaluate Model
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc_score = roc_auc_score(y_test, y_pred)

# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1-Score: {f1:.4f}")
# print(f"AUC-ROC Score: {auc_score:.4f}")

## Hyperparameter

### Optimal CV

In [ ]:
# from sklearn.model_selection import cross_val_score
# import numpy as np

# cv_values = [2, 3, 5, 10]  # Coba berbagai nilai cv
# scores = []

# for cv in cv_values:
#     cv_score = cross_val_score(base_model_pipe, 
#                                X_train, 
#                                y_train, 
#                                cv = cv, 
#                                scoring = 'accuracy')
    
#     mean_score = np.mean(cv_score)
#     std_score = np.std(cv_score)
    
#     scores.append((cv, mean_score, std_score))
#     print(f"Cross Validation: {cv} -> Mean: {mean_score:.4f}, Std: {std_score:.4f}")

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Coba berbagai nilai k
cv_values = [3, 5, 7, 10]
results = {}

for k in cv_values:
    cv_strategy = StratifiedKFold(n_splits = k, shuffle = True, random_state = 42)
    scores = cross_val_score(base_model_pipe, X, y, cv = cv_strategy, scoring = 'accuracy')
    results[k] = (np.mean(scores), np.std(scores))  # Simpan rata-rata dan standar deviasi

# Cetak hasil
for k, (mean_score, std_dev) in results.items():
    print(f"CV={k}: Mean Accuracy={mean_score:.4f}, Std Dev={std_dev:.4f}")

In [ ]:
import matplotlib.pyplot as plt

k_values = list(results.keys())
mean_scores = [results[k][0] for k in k_values]
std_devs = [results[k][1] for k in k_values]

plt.figure(figsize=(8,5))
plt.errorbar(k_values, mean_scores, yerr=std_devs, fmt='o-', capsize=5, label='Mean Accuracy ± Std Dev')
plt.xlabel('Number of CV Folds (k)')
plt.ylabel('Accuracy')
plt.title('Finding Optimal CV for Cross Validation')
plt.legend()
plt.grid()
plt.show()


### Randomsearch

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# import matplotlib.pyplot as plt

# n_iter_values = [10, 20, 50, 100]  # Uji berbagai nilai
# scores = []

# param_grid = {
#     "model__n_estimators": [50, 100, 200, 500],
#     "model__max_depth": [None, 10, 20, 30],
#     "model__min_samples_split": [2, 5, 10, 20],
#     "model__min_samples_leaf": [1, 2, 5, 10],
#     "model__max_features": ["sqrt", "log2", None],
#     "model__bootstrap": [True, False],
#     "model__class_weight": ["balanced", "balanced_subsample", None]
# }

# for n in n_iter_values:
#     random_search = RandomizedSearchCV(
#         base_model_pipe, 
#         param_grid, 
#         n_iter = n, 
#         cv = 3, 
#         scoring = "recall", 
#         n_jobs = -1, 
#         random_state = 42
#     )

#     random_search.fit(X_train, y_train)
#     scores.append(random_search.best_score_)

# plt.plot(n_iter_values, scores, marker='o')
# plt.xlabel("n_iter")
# plt.ylabel("Best CV Score")
# plt.title("Evaluasi n_iter di RandomizedSearchCV")
# plt.show()

In [ ]:
# random_search = RandomizedSearchCV(
#     base_model_pipe,    # Model pipeline
#     param_distributions = param_grid,   # Gunakan param_grid yang sama
#     n_iter = 20,    # Batasi jumlah kombinasi yang diuji (misal 20)
#     cv = 5,     # Cross-validation 5-fold
#     scoring = ["precision", "recall", "f1"], 
#     refit = "recall",
#     n_jobs = -1,    # Gunakan semua core CPU
#     verbose = 2,    # Tampilkan progress
#     random_state = 42   # Pastikan hasil tetap reproducible
# )

# random_search.fit(X_train, y_train)

# best_model = random_search.best_estimator_


In [ ]:
# # Pilih model terbaik
# random_results = random_search.cv_results_
# random_results.keys()

# print("Best parameters based on recall:", random_search.best_params_)

In [ ]:
# # Cek metrik lainnya jika menggunakan multiple scoring
# print("Best model based on precision:", np.max(random_results['mean_test_precision']))
# print("Best model based on F1-score:", np.max(random_results['mean_test_f1']))

In [ ]:
# # Evaluasi Model Terbaik
# y_pred_best = best_model.predict(X_test)

# print("Classification Report:\n", classification_report(y_test, y_pred_best))

In [ ]:
# # Metrik tambahan
# accuracy = accuracy_score(y_test, y_pred_best)
# precision = precision_score(y_test, y_pred_best)
# recall = recall_score(y_test, y_pred_best)
# f1 = f1_score(y_test, y_pred_best)
# auc_score = roc_auc_score(y_test, y_pred_best)

# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1-Score: {f1:.4f}")
# print(f"AUC-ROC Score: {auc_score:.4f}")

### Halvingsearch

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {
    "model__n_estimators": [50, 100, 200, 500],
    "model__max_depth": [None, 10, 20, 30],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [1, 2, 5, 10],
    "model__max_features": ["sqrt", "log2", None],
    "model__bootstrap": [True, False],
    "model__class_weight": ["balanced", "balanced_subsample", None]
}

halving_search = HalvingGridSearchCV(
    base_model_pipe,    # Model pipeline
    param_grid,     # Gunakan param_grid yang sama
    factor = 3,     # Kandidat dikurangi 3x di setiap iterasi
    cv = 3,     # Cross-validation lebih kecil untuk kecepatan
    scoring = "recall",    # Single Metrik 
    refit = True, 
    n_jobs = -1,    # Gunakan semua core CPU
    verbose = 2,    # Tampilkan progress
    random_state = 42   # Untuk hasil yang konsisten
)

halving_search.fit(X_train, y_train)

best_model = halving_search.best_estimator_

# more than 1/2 hour

In [ ]:
# Pilih model terbaik berdasarkan akurasi
halving_results = halving_search.cv_results_
halving_results.keys()

print("Best parameters based on recall:", halving_search.best_params_)

In [ ]:
# Cek metrik lainnya jika menggunakan multiple scoring
print("Best model based on precision:", np.max(halving_results['mean_test_precision']))
print("Best model based on F1-score:", np.max(halving_results['mean_test_f1']))

In [ ]:
# Evaluasi Model Terbaik
y_pred_best = best_model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred_best))

In [ ]:
# Metrik tambahan
accuracy = accuracy_score(y_test, y_pred_best)
precision = precision_score(y_test, y_pred_best)
recall = recall_score(y_test, y_pred_best)
f1 = f1_score(y_test, y_pred_best)
auc_score = roc_auc_score(y_test, y_pred_best)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC Score: {auc_score:.4f}")

# Sample 2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE

# Simulasi data (gantilah dengan dataset asli)
X = pd.DataFrame(np.random.randn(226969, 10), columns=[f'feature_{i}' for i in range(10)])
y = np.array([0] * 223386 + [1] * 3583)  # Imbalanced dataset (fraud cases sangat sedikit)

# Stratified train-test split agar distribusi kelas tetap terjaga
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Transformasi
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=True, max_categories=50)
ordinal_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Kolom kategori (gantilah sesuai dataset Anda)
numeric_cols = [f'feature_{i}' for i in range(10)]  # Semua dianggap numerik
one_hot_cols = []  # Tambahkan jika ada fitur kategori one-hot
ordinal_cols = []  # Tambahkan jika ada fitur ordinal

# Column Transformer
prep_stage_2 = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numeric_cols),
        ("cat", categorical_transformer, one_hot_cols),
        ("ord", ordinal_transformer, ordinal_cols)
    ], remainder="passthrough"
)

# SMOTETomek untuk menangani data imbalanced
smote = SMOTE(sampling_strategy=0.8, random_state=42)
tomek = TomekLinks(sampling_strategy='majority')
sampling = SMOTETomek(smote=smote, tomek=tomek, random_state=42)

# Tangani kasus ZeroDivisionError jika kelas minoritas tidak ada di y_train
if np.sum(y_train == 1) == 0:
    scale_pos_weight = 1
else:
    scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)

# Pipeline dengan model XGBoost
xgb_pipe = Pipeline([
    ("preprocessor", prep_stage_2),
    ("sampling", sampling),
    ("model", XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42
    ))
])

# K-Fold Cross Validation dengan StratifiedKFold
k_values = [3, 5, 7, 10]
scores = []

for k in k_values:
    cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    score = np.mean(cross_val_score(xgb_pipe, X_train, y_train, cv=cv, scoring='accuracy'))
    scores.append((k, score))
    print(f"K = {k}: Mean Cross-Val Score = {score:.4f}")

# Menentukan nilai K terbaik
best_k = sorted(scores, key=lambda x: x[1], reverse=True)[0][0]
print(f"Optimal K untuk Cross-Validation: {best_k}")


# Sample 3

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from collections import Counter

# Oversampling dengan SMOTE (hanya meningkatkan minoritas hingga 30% dari mayoritas)
smote = SMOTE(sampling_strategy=0.3, random_state=42)

# Alternatif undersampling yang lebih hati-hati dengan NearMiss
undersample = NearMiss(version=3, n_neighbors=3)  # Mengurangi mayoritas tanpa kehilangan pola penting

# Lakukan resampling hanya dengan SMOTE terlebih dahulu
X_train_resample, y_train_resample = smote.fit_resample(X_train_tf, y_train)

# (Opsional) Jika ingin menggunakan undersampling setelah SMOTE
# X_train_resample, y_train_resample = undersample.fit_resample(X_train_resample, y_train_resample)

# Print distribusi sebelum & sesudah resampling
print("Before Resampling:")
print(y_train.value_counts(normalize=True) * 100)

print("\nAfter Resampling:")
print(y_train_resample.value_counts(normalize=True) * 100)

# Cek jumlah sampel setelah resampling
print(f"\nOriginal dataset shape: {Counter(y_train)}")
print(f"Resampled dataset shape: {Counter(y_train_resample)}")


# Sample 4

optuna (bayesian optimization)

In [ ]:
# import optuna
# from optuna.integration import SklearnPruningCallback
# from sklearn.model_selection import cross_val_score

# def objective(trial):
#     params = {}
    
#     if final_model_name == "Random Forest":
#         params = {
#             'n_estimators': trial.suggest_int('n_estimators', 100, 500),
#             'max_depth': trial.suggest_int('max_depth', 5, 30),
#             'min_samples_split': trial.suggest_int('min_samples_split', 2, 10)
#         }

#     elif final_model_name == "XGBoost":
#         params = {
#             'n_estimators': trial.suggest_int('n_estimators', 100, 500),
#             'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
#             'max_depth': trial.suggest_int('max_depth', 3, 10)
#         }

#     elif final_model_name == "LightGBM":
#         params = {
#             'num_leaves': trial.suggest_int('num_leaves', 31, 200),
#             'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
#             'n_estimators': trial.suggest_int('n_estimators', 100, 500)
#         }

#     elif final_model_name == "CatBoost":
#         params = {
#             'iterations': trial.suggest_int('iterations', 100, 500),
#             'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
#             'depth': trial.suggest_int('depth', 4, 10)
#         }

#     model = final_model.set_params(**params)
    
#     cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
#     scores = cross_val_score(model, X_train_mod, y_train_mod, cv=cv, scoring='roc_auc')
    
#     return np.mean(scores)

# # Optuna study
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=20, n_jobs=-1)

# best_params = study.best_params
# final_model = final_model.set_params(**best_params)

# print(f"\n🔍 Hyperparameter Terbaik ({final_model_name}): {best_params}")
# print(f"✅ Best ROC-AUC Score: {study.best_value:.3f}")


# Sample 5

In [ ]:
# search = HalvingGridSearchCV(
#     model, 
#     param_grid, 
#     factor=2,  # Setiap iterasi mengeliminasi setengah kandidat
#     scoring='roc_auc',
#     cv=5,  # Cross Validation 5-fold
#     verbose=1,
#     n_jobs=-1
# )

# search.fit(X_train_mod, y_train_mod)


# Sample 6

### Re-Sampling

In [ ]:
# # over-sampling
# smote = SMOTE(sampling_strategy = 0.8, 
#               k_neighbors = NearestNeighbors(n_jobs = -1), 
#               random_state = 42)

# # under-sampling
# tomek = TomekLinks(sampling_strategy = 'majority')

# # resampling
# sampling = SMOTETomek(smote = smote, 
#                       tomek = tomek, 
#                       random_state = 42)

# X_train_resample, y_train_resample = sampling.fit_resample(X_train_tf, y_train)

In [ ]:
# # over-sampling
# smote = SMOTE(sampling_strategy = 0.4, 
#               k_neighbors = NearestNeighbors(n_jobs = -1), 
#               random_state = 42)

# # under-sampling
# tomek = TomekLinks(sampling_strategy = 'auto')

# # resampling
# sampling = SMOTETomek(smote = smote, 
#                       tomek = tomek, 
#                       random_state = 42)

# X_train_resample, y_train_resample = sampling.fit_resample(X_train_tf, y_train)

# Note

Pemilihan sampling data sangat penting ketika data mengalami ketidak seimbangan (imbalanced data), pemilihan model resampling juga akan menentukan dan dapat berpengaruh terhadap overfitting, pilihlah sesuai resample yang dibutuhkan

Setelah melalui proses tersebut maka langkah berikutnya adalah pemilihan model: baik secara manual yakni menggunakan pipleine atau bisa cara menggabungkan: voting atau stacking

langkah berikutnya adalah memilih metrics yang tepat dalam mengatasi permasalahan yang ada, misal untuk fraud lebih sering menggunakan Average Precision (AP) Score atau Precision-Recall Curve

| Metode Evaluasi        | Kode Scoring       | Keterangan |
|------------------------|--------------------|------------|
| **PR-AUC**            | `average_precision` | ✅ Terbaik untuk dataset imbalance |
| **Recall**            | `recall`            | 🔥 Terbaik jika ingin menangkap sebanyak mungkin fraud |
| **F1-Score**          | `f1`                | ⚖ Keseimbangan antara menangkap fraud & menghindari false positives |
| **Precision**         | `precision`         | 🛑 Jika ingin menghindari false positives |
| **ROC-AUC**          | `roc_auc`           | ⚠ Kurang direkomendasikan jika fraud sangat jarang |

# backup

## Pipeline Blueprint

In [ ]:
# Rename var train
X_train_mod = X_train_leak.copy()
y_train_mod = y_train_resample.copy()

# Rename var test
X_test_mod = X_test_leak.copy()
y_test_mod = y_test.copy()

### Model Selections

#### Logistic Regression

In [ ]:
logreg_model = LogisticRegression(solver = "liblinear", 
                                #   class_weight = "balanced",
                                  C = 0.5,  # Tambahkan regularisasi
                                  penalty = "l2",
                                  random_state = 42)

#### Decission Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(max_depth = 5, 
                                #   class_weight = "balanced", 
                                  random_state = 42)

#### Random Forest

In [ ]:
forest_model = RandomForestClassifier(n_estimators = 100,  # Kurangi jumlah pohon
                                      max_depth = 5,  # Kurangi kedalaman pohon
                                      min_samples_split = 10,  # Cegah overfitting
                                    #   class_weight = "balanced",
                                      random_state = 42)

#### XGBoost

In [ ]:
# Tangani kasus ZeroDivisionError jika kelas minoritas tidak ada di y_train
if np.sum(y_train_resample == 1) == 0:
    scale_pos_weight = 1
    
else:
    scale_pos_weight = np.sum(y_train_resample == 0) / np.sum(y_train_resample == 1)

In [ ]:
xgb_model = XGBClassifier(n_estimators = 500, 
                        #   scale_pos_weight = scale_pos_weight,
                          learning_rate = 0.05, 
                          max_depth = 3,  
                          subsample = 0.8, 
                          colsample_bytree = 0.8,  
                          reg_alpha = 0.1,  # Regularisasi L1
                          reg_lambda = 0.5,  # Regularisasi L2
                          eval_metric = "aucpr",
                          random_state = 42)

#### LightBGM

In [ ]:
lbgm_model = LGBMClassifier(is_unbalance = True, 
                            force_col_wise = True, 
                            max_depth = 10,  # Menambah kedalaman pohon 
                            min_data_in_leaf = 10,  # Menghindari split yang tidak berguna 
                            eval_metric = "logloss",  # Metode evaluasi yang lebih jelas 
                            verbose = -1,  # Mengurangi log yang berlebihan 
                            random_state = 42)

#### CatBoost

In [ ]:
catb_model = CatBoostClassifier(verbose = 0, 
                                # auto_class_weights = 'Balanced', 
                                random_state = 42)

#### Pipeline Methode

In [ ]:
from sklearn.metrics import average_precision_score, classification_report, precision_recall_curve
import matplotlib.pyplot as plt

# Menyimpan semua pipeline dalam dictionary
pipelines = {
    "Logistic Regression": logreg_model,
    "Decission Tree": dt_model, 
    "Random Forest": forest_model,
    "XGBoost": xgb_model,
    "LightGBM": lbgm_model,
    "CatBoost": catb_model
}

# Evaluasi semua model
best_ap_test = 0  
best_score_diff = float("inf")  
best_model = None
best_model_name = ""

for name, pipe in pipelines.items():
    print(f"🔹 Evaluasi Model: {name}")
    print('=' * 50)
    
    # Training model
    if name == "CatBoost":
        pipe.fit(X_train_mod, y_train_mod, verbose=False)
    else:
        pipe.fit(X_train_mod, y_train_mod)
    
    # === Train Evaluation ===
    y_train_pred_proba = pipe.predict_proba(X_train_mod)[:, 1]
    ap_train = average_precision_score(y_train_mod, y_train_pred_proba)
    print(f"📊 Average Precision Score (Train): {ap_train:.3f}")

    # === Test Evaluation ===
    y_test_pred_proba = pipe.predict_proba(X_test_mod)[:, 1]
    ap_test = average_precision_score(y_test_mod, y_test_pred_proba)
    
    # Filter model jika AP Score terlalu tinggi
    if ap_test > 0.95:
        print(f"⚠️ AP Score Test terlalu tinggi ({ap_test:.3f}), model ini diabaikan.\n")
        continue  

    print(f"📊 Average Precision Score (Test): {ap_test:.3f}")

    # Precision-Recall Curve
    precisions, recalls, thresholds = precision_recall_curve(y_test_mod, y_test_pred_proba)
    
    # Cari threshold optimal
    f1_scores = (2 * precisions * recalls) / (precisions + recalls + 1e-9)  # Hindari divisi oleh nol
    best_threshold = thresholds[f1_scores.argmax()] if len(thresholds) > 0 else 0.5
    print(f"🎯 Optimal Threshold: {best_threshold:.3f}")

    # Konversi probabilitas menjadi label menggunakan threshold optimal
    y_test_pred = (y_test_pred_proba >= best_threshold).astype(int)

    # Classification Report
    print("\n📄 Classification Report:")
    print(classification_report(y_test_mod, y_test_pred))

    print('=' * 50, '\n')

    # Evaluasi model terbaik berdasarkan AP Score Test dan perbedaan train-test terkecil
    score_diff = abs(ap_train - ap_test)

    if ap_test > best_ap_test or (ap_test == best_ap_test and score_diff < best_score_diff):
        best_model = pipe
        best_model_name = name
        best_ap_test = ap_test
        best_score_diff = score_diff

# Menampilkan hasil akhir
if best_model is not None:
    print(f"🏆 Model Terbaik: {best_model_name} dengan AP Score Test tertinggi: {best_ap_test:.3f} dan perbedaan AP Score: {best_score_diff:.3f}")
else:
    print("❌ Tidak ada model yang memenuhi syarat (AP Score Test ≤ 0.95).")


🔹 Evaluasi Model: Logistic Regression
📊 Average Precision Score (Train): 0.962
📊 Average Precision Score (Test): 0.927
🎯 Optimal Threshold: 0.479

📄 Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56549
           1       0.87      0.90      0.88      2369

    accuracy                           0.99     58918
   macro avg       0.93      0.95      0.94     58918
weighted avg       0.99      0.99      0.99     58918


🔹 Evaluasi Model: Decission Tree
📊 Average Precision Score (Train): 1.000
⚠️ AP Score Test terlalu tinggi (1.000), model ini diabaikan.

🔹 Evaluasi Model: Random Forest
📊 Average Precision Score (Train): 1.000
⚠️ AP Score Test terlalu tinggi (1.000), model ini diabaikan.

🔹 Evaluasi Model: XGBoost
📊 Average Precision Score (Train): 1.000
⚠️ AP Score Test terlalu tinggi (1.000), model ini diabaikan.

🔹 Evaluasi Model: LightGBM
📊 Average Precision Score (Train): 1.000
⚠️ AP Score Test terlalu ting

#### Voting Methode

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import average_precision_score, classification_report, precision_recall_curve

# Membuat Voting Classifier dengan model terbaik
voting_clf = VotingClassifier(
    estimators=[
        ('rf', forest_model),
        ('dt', dt_model), 
        ('xgb', xgb_model),
        ('lgbm', lbgm_model),
        ('catb', catb_model)
    ],
    voting='soft'  # Menggunakan probabilitas
)

# Training Voting Classifier
voting_clf.fit(X_train_mod, y_train_mod)

# Evaluasi pada Test Set
y_test_pred_proba_voting = voting_clf.predict_proba(X_test_mod)[:, 1]

# Hitung Average Precision Score (AP Score) untuk Voting Classifier
best_ap_voting = average_precision_score(y_test_mod, y_test_pred_proba_voting)

# Filter jika AP Score Voting Classifier terlalu tinggi
if best_ap_voting > 0.95:
    print(f"⚠️ AP Score Voting Classifier terlalu tinggi ({best_ap_voting:.3f}), model ini diabaikan.")
    best_ap_voting = 0  # Abaikan model ini dari pemilihan final
else:
    print(f"📊 Average Precision Score (Voting Classifier): {best_ap_voting:.3f}")

    # Precision-Recall Curve
    precisions, recalls, thresholds = precision_recall_curve(y_test_mod, y_test_pred_proba_voting)

    # Cari threshold optimal
    f1_scores = (2 * precisions * recalls) / (precisions + recalls + 1e-9)  # Hindari divisi oleh nol
    best_threshold_voting = thresholds[f1_scores.argmax()] if len(thresholds) > 0 else 0.5
    print(f"🎯 Optimal Threshold: {best_threshold_voting:.3f}")

    # Konversi probabilitas menjadi label menggunakan threshold optimal
    y_test_pred_voting = (y_test_pred_proba_voting >= best_threshold_voting).astype(int)

    # Evaluasi Voting Classifier
    print("\n=== Classification Report (Voting Classifier) ===")
    print(classification_report(y_test_mod, y_test_pred_voting)) 


⚠️ AP Score Voting Classifier terlalu tinggi (1.000), model ini diabaikan.


### Choosing Best Model

In [ ]:
final_model = None
final_model_name = ""
final_score = 0

if best_ap_voting > best_ap_test:
    final_model = voting_clf
    final_model_name = "Voting Classifier"
    final_score = best_ap_voting
elif best_ap_test > 0:
    final_model = best_model
    final_model_name = best_model_name
    final_score = best_ap_test

# Menampilkan model terbaik
if final_model is not None:
    print(f"✅ Model Terbaik untuk CV: {final_model_name} dengan Average Precision Score: {final_score:.3f}")
else:
    print("❌ Tidak ada model yang memenuhi syarat (AP Score Test ≤ 0.95).")

✅ Model Terbaik untuk CV: Logistic Regression dengan Average Precision Score: 0.927


## Finding Optimal CV

In [ ]:
# Daftar nilai CV yang ingin diuji
cv_values = [3, 5, 7, 10]
testing_best_cv = None
testing_cv_score = 0

print("🔍 Mencari Nilai CV Optimal dengan StratifiedKFold...")

for cv in cv_values:
    print(f"\nEvaluasi dengan cv = {cv}")
    
    # Gunakan StratifiedKFold
    stratified_cv = StratifiedKFold(n_splits = cv, shuffle = True, random_state = 42)
    print(f"stratified_cv: {stratified_cv}")
    
    scores = cross_val_score(final_model, 
                             X_train_mod, 
                             y_train_mod, 
                             cv = stratified_cv, 
                             scoring = 'average_precision', 
                             n_jobs = -1)
    
    mean_score = np.mean(scores)  # ✅ Gunakan rata-rata, bukan max
    print(f"average_precision rata-rata: {mean_score:.3f} (dengan cv = {cv})")
    
    # Simpan nilai CV terbaik
    if mean_score > testing_cv_score:
        testing_cv_score = mean_score
        testing_best_cv = cv

print(f"\n✅ Nilai CV Optimal: {testing_best_cv} dengan ROC-AUC: {testing_cv_score:.3f}")

🔍 Mencari Nilai CV Optimal dengan StratifiedKFold...

Evaluasi dengan cv = 3
stratified_cv: StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
average_precision rata-rata: 0.962 (dengan cv = 3)

Evaluasi dengan cv = 5
stratified_cv: StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
average_precision rata-rata: 0.962 (dengan cv = 5)

Evaluasi dengan cv = 7
stratified_cv: StratifiedKFold(n_splits=7, random_state=42, shuffle=True)
average_precision rata-rata: 0.962 (dengan cv = 7)

Evaluasi dengan cv = 10
stratified_cv: StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
average_precision rata-rata: 0.962 (dengan cv = 10)

✅ Nilai CV Optimal: 3 dengan ROC-AUC: 0.962


# Hyperparameter Tuning

halving

In [ ]:
# Pilih model yang akan di-tuning
if final_model_name == "Random Forest":
    param_dist = {
        'n_estimators': [100, 300, 500, 1000],  # Jumlah pohon dalam hutan
        'max_depth': [10, 20, 30, None],  # Kedalaman maksimum pohon
        'min_samples_split': [2, 5, 10, 20],  # Minimum sampel untuk melakukan split
        'min_samples_leaf': [1, 2, 5, 10],  # Minimum sampel di setiap daun
        'max_features': ['sqrt', 'log2', None],  # Fitur yang dipertimbangkan per split
        'bootstrap': [True, False]  # Apakah menggunakan bootstrap sampling
    }

elif final_model_name == "Logistic Regression":
    param_dist = {
        'penalty': ['l1', 'l2', 'elasticnet', None],  # Jenis regularisasi
        'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Parameter inversi dari regularisasi
        'solver': ['liblinear', 'lbfgs', 'saga', 'newton-cg'],  # Algoritma optimasi
        'max_iter': [100, 200, 500, 1000],  # Jumlah iterasi maksimum
        'l1_ratio': [0, 0.1, 0.5, 0.9, 1]  # Hanya berlaku untuk elasticnet
    }


elif final_model_name == "XGBoost":
    param_dist = {
        'n_estimators': [100, 200, 300, 500],  # Jumlah pohon boosting
        'learning_rate': [0.001, 0.01, 0.1, 0.2],  # Laju pembelajaran
        'max_depth': [3, 6, 10, 15],  # Kedalaman maksimum pohon
        'min_child_weight': [1, 3, 5, 7],  # Bobot minimum anak untuk split
        'subsample': [0.6, 0.8, 1.0],  # Rasio sampel yang digunakan dalam training
        'colsample_bytree': [0.6, 0.8, 1.0],  # Proporsi fitur yang digunakan per pohon
        'gamma': [0, 0.1, 0.2, 0.5],  # Pengendalian pruning dengan minimum loss reduction
        'reg_lambda': [0, 0.1, 1, 10],  # Regularisasi L2
        'reg_alpha': [0, 0.1, 1, 10]  # Regularisasi L1
    }

elif final_model_name == "LightGBM":
    param_dist = {
        'num_leaves': [31, 50, 100, 150],  # Jumlah daun pada setiap pohon
        'learning_rate': [0.001, 0.01, 0.1, 0.2],  # Laju pembelajaran
        'n_estimators': [100, 200, 300, 500],  # Jumlah pohon boosting
        'max_depth': [-1, 10, 20, 30],  # Kedalaman maksimum pohon (-1 berarti tidak terbatas)
        'min_child_samples': [10, 20, 50, 100],  # Minimum sampel dalam satu leaf
        'subsample': [0.6, 0.8, 1.0],  # Proporsi sampel yang digunakan
        'colsample_bytree': [0.6, 0.8, 1.0],  # Proporsi fitur yang digunakan per pohon
        'reg_lambda': [0, 0.1, 1, 10],  # Regularisasi L2
        'reg_alpha': [0, 0.1, 1, 10]  # Regularisasi L1
    }

elif final_model_name == "CatBoost":
    param_dist = {
        'iterations': [100, 200, 300, 500],  # Jumlah iterasi boosting
        'learning_rate': [0.001, 0.01, 0.1, 0.2],  # Laju pembelajaran
        'depth': [4, 6, 10, 12],  # Kedalaman maksimum pohon
        'l2_leaf_reg': [1, 3, 5, 10],  # Regularisasi L2 untuk leaf
        'bagging_temperature': [0.1, 0.5, 1, 2],  # Kontrol bootstraping (mirip dengan subsample)
        'border_count': [32, 64, 128],  # Jumlah bin untuk fitur numerik
        'random_strength': [0.1, 0.5, 1, 2]  # Noise untuk regularisasi
    }

In [ ]:
# from itertools import product

# # Pastikan `final_model_name` memiliki nilai yang valid
# if "final_model_name" not in globals():
#     raise ValueError("Variabel 'final_model_name' belum didefinisikan!")

# # Pilih model yang akan di-tuning
# if final_model_name == "Random Forest":
#     param_dist = {
#         'n_estimators': [100, 300, 500, 1000],
#         'max_depth': [10, 20, 30, None],
#         'min_samples_split': [2, 5, 10, 20],
#         'min_samples_leaf': [1, 2, 5, 10],
#         'max_features': ['sqrt', 'log2', None],
#         'bootstrap': [True, False]
#     }

# elif final_model_name == "XGBoost":
#     param_dist = {
#         'n_estimators': [100, 200, 300, 500],
#         'learning_rate': [0.001, 0.01, 0.1, 0.2],
#         'max_depth': [3, 6, 10, 15],
#         'min_child_weight': [1, 3, 5, 7],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'gamma': [0, 0.1, 0.2, 0.5],
#         'reg_lambda': [0, 0.1, 1, 10],
#         'reg_alpha': [0, 0.1, 1, 10]
#     }

# elif final_model_name == "LightGBM":
#     param_dist = {
#         'num_leaves': [31, 50, 100, 150],
#         'learning_rate': [0.001, 0.01, 0.1, 0.2],
#         'n_estimators': [100, 200, 300, 500],
#         'max_depth': [-1, 10, 20, 30],
#         'min_child_samples': [10, 20, 50, 100],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'reg_lambda': [0, 0.1, 1, 10],
#         'reg_alpha': [0, 0.1, 1, 10]
#     }

# elif final_model_name == "CatBoost":
#     param_dist = {
#         'iterations': [100, 200, 300, 500],
#         'learning_rate': [0.001, 0.01, 0.1, 0.2],
#         'depth': [4, 6, 10, 12],
#         'l2_leaf_reg': [1, 3, 5, 10],
#         'bagging_temperature': [0.1, 0.5, 1, 2],
#         'border_count': [32, 64, 128],
#         'random_strength': [0.1, 0.5, 1, 2]
#     }

# else:
#     raise ValueError(f"Model '{final_model_name}' tidak dikenali!")


In [ ]:
# # 🔹 Pastikan X_train_mod adalah DataFrame
# if isinstance(X_train_mod, np.ndarray):
#     X_train_mod = pd.DataFrame(X_train_mod)

# # 🔹 Bersihkan dataset dari NaN
# X_train_mod = X_train_mod.fillna(X_train_mod.median())

# # 🔹 Bersihkan dataset dari Inf
# X_train_mod = X_train_mod.replace([np.inf, -np.inf], np.nan)  # Ganti Inf jadi NaN
# X_train_mod = X_train_mod.fillna(X_train_mod.median())  # Isi ulang dengan median

# # 🔹 Konversi kembali ke NumPy jika diperlukan
# X_train_mod = X_train_mod.to_numpy()

In [ ]:
# 🔹 Hitung jumlah kombinasi parameter jika `param_dist` sudah terdefinisi
if "param_dist" in globals():
    total_combinations = len(list(product(*param_dist.values())))
    print(f"Total kombinasi parameter valid: {total_combinations}")

    # 🔹 Atur `n_candidates` agar tidak lebih besar dari jumlah kombinasi parameter
    param_combinations = min(100, total_combinations)
else:
    raise ValueError("param_dist tidak terdefinisi.")

Total kombinasi parameter valid: 1920


In [ ]:
# # Konversi kembali ke DataFrame dengan kolom asli
# X_train_mod = pd.DataFrame(X_train_mod, columns=X_train.columns)
# X_test_mod = pd.DataFrame(X_test_mod, columns=X_test.columns)

# # 🔍 Cek dan Tangani NaN & Inf
# def clean_data(df):
#     # Simpan nama kolom
#     columns = df.columns

#     # Pastikan semua kolom numerik
#     df = df.apply(pd.to_numeric, errors='coerce')

#     # Cek apakah ada NaN atau Inf
#     if np.any(pd.isnull(df)) or np.any(np.isinf(df)):
#         print("⚠️ Warning: Dataset mengandung NaN atau Inf! Membersihkannya...")

#         # Ganti NaN dengan median setiap kolom
#         df = df.fillna(df.median())

#         # Ganti Inf dengan nilai maksimum di setiap kolom
#         df = df.replace([np.inf, -np.inf], df.max())

#     # Pastikan DataFrame tetap memiliki nama kolom
#     df.columns = columns

#     return df

# # 🔹 Bersihkan X_train_mod & X_test_mod
# X_train_mod = clean_data(X_train_mod)
# X_test_mod = clean_data(X_test_mod)

# # 🔍 Cek statistik dataset setelah pembersihan
# print(X_train_mod.describe())


In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

# 
skf = StratifiedKFold(n_splits = testing_best_cv, shuffle = True, random_state = 42)

# 🔍 Hyperparameter Tuning dengan HalvingRandomSearchCV
halving_search = HalvingRandomSearchCV(
    final_model, 
    param_distributions = param_dist, 
    factor = 2, # Total combination / factor
    scoring = 'average_precision', 
    cv = skf, 
    n_jobs = -1, 
    random_state = 42,
    n_candidates = param_combinations, 
    error_score = "raise"
)

halving_search.fit(X_train_mod, y_train_mod)

print(f"\n🔍 Hyperparameter Terbaik ({final_model_name}): {halving_search.best_params_}")
print(f"✅ Best ROC-AUC Score: {halving_search.best_score_:.3f}")

optimal_cv = halving_search.best_estimator_

ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

# Final Evaluation

In [ ]:
from sklearn.metrics import average_precision_score, classification_report, precision_recall_curve, roc_auc_score

# 🔹 Konversi ke NumPy untuk menghindari warning
X_test_mod = X_test_mod.to_numpy()

# 🔹 Prediksi dengan Model Terbaik
y_test_pred_proba = optimal_cv.predict_proba(X_test_mod)[:, 1]

# 🔹 Menghitung Average Precision Score (AP Score)
ap_score = average_precision_score(y_test_mod, y_test_pred_proba)
print(f"📊 Average Precision Score (Test): {ap_score:.3f}")

# 🔹 Mencari Threshold Optimal menggunakan Precision-Recall Curve
precisions, recalls, thresholds = precision_recall_curve(y_test_mod, y_test_pred_proba)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-9)  # Hindari pembagian oleh nol
best_threshold = thresholds[f1_scores.argmax()] if len(thresholds) > 0 else 0.5

# 🔹 Gunakan Threshold Optimal untuk Prediksi Akhir
y_test_pred = (y_test_pred_proba >= best_threshold).astype(int)

# 🔹 Classification Report
print("\n=== Classification Report ===")
print(classification_report(y_test_mod, y_test_pred))

# 🔹 ROC-AUC Score
roc_auc_final = roc_auc_score(y_test_mod, y_test_pred_proba)
print(f"🎯 Final ROC-AUC Score: {roc_auc_final:.3f}")
print(f"🔹 Best Threshold Used: {best_threshold:.3f}")
